In [1]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from collections import defaultdict

In [2]:
# Root directory of the data
prefix_files = "../berkabank/raw/"

# Information about the data
link = "https://sorry.vse.cz/~berka/challenge/pkdd1999/berka.htm"

# To divide the data into two parts, we will use the following logic:
ref_date = datetime.datetime(day=1, month=1, year=1997)
ref_date

datetime.datetime(1997, 1, 1, 0, 0)

In [3]:
account = pd.read_csv(f"{prefix_files}account.csv", sep=";")
# Convert date to datetime
account["date"] = pd.to_datetime(account["date"], format="%y%m%d")
# convert account frequency to english
freq_options = {
    "POPLATEK MESICNE": "monthly",
    "POPLATEK TYDNE": "weekly",
    "POPLATEK PO OBRATU": "transaction",
}

account["frequency"] = account["frequency"].map(freq_options).fillna("unknown")


# Rename columns
account.rename(
    columns={
        "date": "account_creation_date",
        "frequency": "account_statement_frequency",
    },
    inplace=True,
)

account.head()

,account_id,district_id,account_statement_frequency,account_creation_date
0,576,55,monthly,1993-01-01
1,3818,74,monthly,1993-01-01
2,704,55,monthly,1993-01-01
3,2378,16,monthly,1993-01-01
4,2632,24,monthly,1993-01-02


In [58]:
card = pd.read_csv(f"{prefix_files}card.csv", sep=";")

# Convert date to datetime
card["issued"] = pd.to_datetime(card["issued"])

# Rename columns
card.rename(columns={"issued": "card_issued_date", "type": "card_type"}, inplace=True)


card = card.sort_values(["card_issued_date", "card_id"])
card.head()

/var/folders/2v/jln6f4j122qcvxp2jjr6cclm0000gn/T/ipykernel_47934/4105393297.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  card['issued'] = pd.to_datetime(card['issued'])


,card_id,disp_id,card_type,card_issued_date
0,1005,9285,classic,1993-11-07
1,104,588,classic,1994-01-19
2,747,4915,classic,1994-02-05
3,70,439,classic,1994-02-08
4,577,3687,classic,1994-02-15


In [5]:
client = pd.read_csv(f"{prefix_files}client.csv", sep=";")

# Drop columns
client.drop(columns=["birth_number"], inplace=True)

client.head()

,client_id,district_id
0,1,18
1,2,1
2,3,1
3,4,5
4,5,5


In [6]:
disp = pd.read_csv(f"{prefix_files}disp.csv", sep=";")

# Lowercase all values in the type column
disp.type = disp.type.str.lower()
# Rename columns
disp.rename(columns={"type": "disp_type"}, inplace=True)
disp.head()

,disp_id,client_id,account_id,disp_type
0,1,1,1,owner
1,2,2,2,owner
2,3,3,2,disponent
3,4,4,3,owner
4,5,5,3,disponent


In [7]:
district = pd.read_csv(f"{prefix_files}district.csv", sep=";")
district = district.rename(
    columns={
        "A1": "district_id",
        "A2": "district_name",
        "A3": "region",
        "A4": "num_inhabitants",
        "A5": "num_munipalities_gt499",
        "A6": "num_munipalities_500to1999",
        "A7": "num_munipalities_2000to9999",
        "A8": "num_munipalities_gt10000",
        "A9": "num_cities",
        "A10": "ratio_urban",
        "A11": "average_salary",
        "A12": "unemp_rate95",
        "A13": "unemp_rate96",
        "A14": "num_entrep_per1000",
        "A15": "num_crimes95",
        "A16": "num_crimes96",
    }
)

# Lowercase all values in the region column
district.region = district.region.str.lower()
district.district_name = district.district_name.str.lower()

# Drop columns
district.drop(
    columns=[
        "num_crimes95",
        "num_crimes96",
        "unemp_rate95",
        "unemp_rate96",
        "num_entrep_per1000",
        "num_munipalities_gt499",
        "num_munipalities_500to1999",
        "num_munipalities_2000to9999",
        "num_munipalities_gt10000",
        "num_cities",
    ],
    inplace=True,
)

district.head()

,district_id,district_name,region,num_inhabitants,ratio_urban,average_salary
0,1,hl.m. praha,prague,1204953,100.0,12541
1,2,benesov,central bohemia,88884,46.7,8507
2,3,beroun,central bohemia,75232,41.7,8980
3,4,kladno,central bohemia,149893,67.4,9753
4,5,kolin,central bohemia,95616,51.4,9307


In [8]:
loan = pd.read_csv(f"{prefix_files}loan.csv", sep=";")

# Map loan status
status_mapping = {
    "A": "paid_no_arrears",
    "B": "defaulted",
    "C": "running_no_arrears",
    "D": "running_with_arrears",
}
loan["status"] = loan["status"].map(status_mapping)

# Rename columns
loan.rename(
    columns={
        "date": "loan_date",
        "amount": "loan_principal",
        "duration": "loan_tenor_months",
        "payments": "loan_monthly_installment",
        "status": "loan_status",
    },
    inplace=True,
)
# Convert date to datetime
loan["loan_date"] = pd.to_datetime(loan["loan_date"], format="%y%m%d")

# Calculate expected end date for loan
days_per_months = 31
loan["estimated_loan_end_date"] = loan["loan_date"] + pd.to_timedelta(
    loan["loan_tenor_months"] * days_per_months, unit="d"
)

loan.head()

,loan_id,account_id,loan_date,loan_principal,loan_tenor_months,loan_monthly_installment,loan_status,estimated_loan_end_date
0,5314,1787,1993-07-05,96396,12,8033.0,defaulted,1994-07-12
1,5316,1801,1993-07-11,165960,36,4610.0,paid_no_arrears,1996-07-31
2,6863,9188,1993-07-28,127080,60,2118.0,paid_no_arrears,1998-08-31
3,5325,1843,1993-08-03,105804,36,2939.0,paid_no_arrears,1996-08-23
4,7240,11013,1993-09-06,274740,60,4579.0,paid_no_arrears,1998-10-10


In [9]:
order = pd.read_csv(f"{prefix_files}order.csv", sep=";")

k_symbol_map = {
    "POJISTNE": "insurance_payment",
    "SLUZBY": "payment_for_statement",
    "UROK": "interest_credited",
    "SANKC. UROK": "sanction_interest",
    "SIPO": "household_payment",
    "DUCHOD": "old_age_pension",
    "UVER": "loan_payment",
    "LEASING": "leasing_payment",
}

order.k_symbol = order.k_symbol.map(k_symbol_map).fillna("unknown")

# Rename columns
order.rename(
    columns={
        "bank_to": "order_bank_to",
        "account_to": "order_account_to",
        "amount": "order_amount",
    },
    inplace=True,
)
order.head()

,order_id,account_id,order_bank_to,order_account_to,order_amount,k_symbol
0,29401,1,YZ,87144583,2452.0,household_payment
1,29402,2,ST,89597016,3372.7,loan_payment
2,29403,2,QR,13943797,7266.0,household_payment
3,29404,3,WX,83084338,1135.0,household_payment
4,29405,3,CD,24485939,327.0,unknown


In [10]:
trans = pd.read_csv(f"{prefix_files}trans.csv", sep=";", low_memory=False)

# Map the operation column to english
operation_map = {
    "VYBER KARTOU": "credit_card_withdrawal",
    "VYBER": "withdrawal",
    "VKLAD": "deposit",
    "PREVOD Z UCTU": "outgoing_transfer_to_bank",
    "PREVOD NA UCET": "incoming_transfer_from_bank",
}
trans.operation = trans.operation.map(operation_map).fillna("unknown")
# Map the type column to english
type_map = {"PRIJEM": "inflow", "VYDAJ": "outflow", "VYBER": "withdrawal"}


# Map the transaction types using the operation mapping dictionary
trans["type"] = trans["type"].map(type_map).fillna("unknown")

# Map the k_symbol column to english
k_symbol_map = {
    "POJISTNE": "insurance_payment",
    "SLUZBY": "payment_for_statement",
    "UROK": "interest_credited",
    "SANKC. UROK": "sanction_interest",
    "SIPO": "household_payment",
    "DUCHOD": "old_age_pension",
    "UVER": "loan_payment",
    "LEASING": "leasing_payment",
}

trans.k_symbol = trans.k_symbol.map(k_symbol_map).fillna("unknown")


# Convert date to datetime
trans["date"] = pd.to_datetime(trans["date"], format="%y%m%d")

# Drop the columns
trans.drop(columns=["bank", "account"], inplace=True)


# Rename columns
trans.rename(
    columns={
        "date": "transaction_date",
        "type": "transaction_type",
        "operation": "transaction_operation",
        "amount": "transaction_amount",
        "balance": "account_balance_after_transaction",
    },
    inplace=True,
)


trans.head()

,trans_id,account_id,transaction_date,transaction_type,transaction_operation,transaction_amount,account_balance_after_transaction,k_symbol
0,695247,2378,1993-01-01,inflow,deposit,700.0,700.0,unknown
1,171812,576,1993-01-01,inflow,deposit,900.0,900.0,unknown
2,207264,704,1993-01-01,inflow,deposit,1000.0,1000.0,unknown
3,1117247,3818,1993-01-01,inflow,deposit,600.0,600.0,unknown
4,579373,1972,1993-01-02,inflow,deposit,400.0,400.0,unknown


In [11]:
path_to_files = "./berkabank/elaboration/"
# Save the dataframes to csv
account.to_csv(f"{path_to_files}account.csv", index=False)
card.to_csv(f"{path_to_files}card.csv", index=False)
client.to_csv(f"{path_to_files}client.csv", index=False)
disp.to_csv(f"{path_to_files}disp.csv", index=False)
district.to_csv(f"{path_to_files}district.csv", index=False)
loan.to_csv(f"{path_to_files}loan.csv", index=False)
order.to_csv(f"{path_to_files}order.csv", index=False)
trans.to_csv(f"{path_to_files}trans.csv", index=False)

## Divide data based on reference date


In [12]:
# Divide the transaction data into past and future
trans_past = trans.loc[trans["transaction_date"] < ref_date, :]
trans_future = trans.loc[trans["transaction_date"] >= ref_date, :]

In [13]:
# Divide data according data into past and future
account_past = account[
    pd.to_datetime(account["account_creation_date"], format="%d-%m-%Y") < ref_date
]
account_future = account[
    pd.to_datetime(account["account_creation_date"], format="%d-%m-%Y") >= ref_date
]
account_past.head()

,account_id,district_id,account_statement_frequency,account_creation_date
0,576,55,monthly,1993-01-01
1,3818,74,monthly,1993-01-01
2,704,55,monthly,1993-01-01
3,2378,16,monthly,1993-01-01
4,2632,24,monthly,1993-01-02


In [14]:
card_past = card.loc[card["card_issued_date"] < ref_date, :]
card_future = card.loc[card["card_issued_date"] >= ref_date, :]

In [15]:
disp_past = disp.merge(account_past, on="account_id").sort_values(
    "account_creation_date"
)
disp_future = disp.merge(account_future, on="account_id").sort_values(
    "account_creation_date"
)

In [16]:
trans_past.merge(disp_past, on="account_id").sort_values("transaction_date")

,trans_id,account_id,transaction_date,transaction_type,transaction_operation,transaction_amount,account_balance_after_transaction,k_symbol,disp_id,client_id,disp_type,district_id,account_statement_frequency,account_creation_date
0,695247,2378,1993-01-01,inflow,deposit,700.0,700.0,unknown,2873,2873,owner,16,monthly,1993-01-01
1,171812,576,1993-01-01,inflow,deposit,900.0,900.0,unknown,693,693,disponent,55,monthly,1993-01-01
2,171812,576,1993-01-01,inflow,deposit,900.0,900.0,unknown,692,692,owner,55,monthly,1993-01-01
3,207264,704,1993-01-01,inflow,deposit,1000.0,1000.0,unknown,845,845,disponent,55,monthly,1993-01-01
4,207264,704,1993-01-01,inflow,deposit,1000.0,1000.0,unknown,844,844,owner,55,monthly,1993-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
533814,3632829,3098,1996-12-31,inflow,unknown,124.0,32490.2,interest_credited,3747,3747,owner,1,monthly,1996-09-16
533813,3630330,3021,1996-12-31,inflow,unknown,122.0,21935.3,interest_credited,3649,3649,owner,10,monthly,1996-04-20
533812,3630965,3040,1996-12-31,inflow,unknown,60.5,15441.1,interest_credited,3674,3674,owner,17,monthly,1996-08-03
533810,3632660,3093,1996-12-31,inflow,unknown,38.4,26008.4,interest_credited,3741,3741,owner,16,monthly,1996-11-21


In [17]:
performance_reached_loans = loan.loc[
    (loan["loan_date"] < ref_date)
    & (loan["loan_status"].isin(["defaulted", "paid_no_arrears"])),
    :,
]

In [18]:
performance_reached_loans.loan_status.value_counts();


loan_status
paid_no_arrears    163
defaulted           28
Name: count, dtype: int64

In [19]:
loan_view = [
    "loan_date",
    "loan_tenor_months",
    "estimated_loan_end_date",
    "loan_principal",
    "loan_status",
]
performance_reached_loans_before_ref = loan.loc[
    loan["estimated_loan_end_date"] < ref_date, loan_view
].sort_values("loan_date")

## End of Day Balance


In [20]:
# Generate list of all dates and account IDs
dates = pd.date_range(
    start=trans["transaction_date"].min(), end=trans["transaction_date"].max(), freq="D"
)
all_account_ids = trans["account_id"].unique()
all_dates_df = pd.DataFrame(
    [(account_id, date) for account_id in all_account_ids for date in dates],
    columns=["account_id", "transaction_date"],
)

# Left join with transactions DataFrame
eod_balance = pd.merge(
    all_dates_df, trans, how="left", on=["account_id", "transaction_date"]
)

# Sort eod_balance by account_id and transaction_date
eod_balance = eod_balance.sort_values(["account_id", "transaction_date"])

# Merge eod_balance with account on account_id to get account_creation_date
eod_balance = eod_balance.merge(
    account[["account_id", "account_creation_date"]], on="account_id", how="left"
)

# Remove transactions before account creation date
eod_balance = eod_balance[
    eod_balance["transaction_date"] >= eod_balance["account_creation_date"]
]

# Fill NaNs with 0
eod_balance["transaction_amount"] = eod_balance["transaction_amount"].fillna(0)


# Give sign to transaction amount based on transaction type
eod_balance["daily_amount_flow"] = np.where(
    eod_balance["transaction_type"] == "outflow",
    -eod_balance["transaction_amount"],
    eod_balance["transaction_amount"],
)

# Calculate end-of-day balance
eod_balance["end_of_day_balance"] = eod_balance.groupby("account_id")[
    "daily_amount_flow"
].cumsum()

# Rename columns
eod_balance.rename(columns={"transaction_date": "balance_date"}, inplace=True)

# Sort by balance_date and account_id
eod_balance = eod_balance.sort_values(["balance_date", "account_id"])

# Filter columns
eod_balance = eod_balance[
    [
        "account_id",
        "balance_date",
        "end_of_day_balance",
        "daily_amount_flow",
        "account_creation_date",
    ]
]

In [21]:
eod_balance

,account_id,balance_date,end_of_day_balance,daily_amount_flow,account_creation_date
1233931,576,1993-01-01,900.0,900.0,1993-01-01
1500144,704,1993-01-01,1000.0,1000.0,1993-01-01
5064621,2378,1993-01-01,700.0,700.0,1993-01-01
8130997,3818,1993-01-01,600.0,600.0,1993-01-01
1233932,576,1993-01-02,900.0,0.0,1993-01-01
...,...,...,...,...,...
10094257,11349,1998-12-31,492736.2,186.0,1995-05-26
10096515,11359,1998-12-31,134259.0,282.6,1994-10-01
10098766,11362,1998-12-31,38530.9,162.8,1995-10-14
10101050,11382,1998-12-31,506198.0,311.3,1995-08-20


## Create Target


In [22]:
# Create a flag if client is primary given the requirements of usage and seniority
transaction_usage_flag = 50
seniority_account_flag = 30
# Each client is non primary by default, we will update this value by the time when requirements are met
eod_balance["is_primary"] = False
# Calculate the number of transactions over time for each account
eod_balance["n_transactions"] = eod_balance.groupby("account_id")[
    "daily_amount_flow"
].cumcount()
# Calculate the number of days since account creation
eod_balance["days_since_account_creation"] = (
    eod_balance["balance_date"] - eod_balance["account_creation_date"]
).dt.days
# Update the is_primary flag
eod_balance.loc[
    (eod_balance["n_transactions"] >= transaction_usage_flag)
    & (eod_balance["days_since_account_creation"] >= seniority_account_flag),
    "is_primary",
] = True

In [39]:
# Create a flag every time the balance is less than target_balance
target_balance = 20000  # 500 EUR
incident_duration_days = 20
# Sort the DataFrame
eod_balance = eod_balance.sort_values(["account_id", "balance_date"])
# Create 'low_balance_flag'
eod_balance["low_balance_flag"] = eod_balance["end_of_day_balance"] < target_balance
# Create 'streak_id'
eod_balance["streak_id"] = (
    eod_balance["low_balance_flag"]
    != eod_balance.groupby("account_id")["low_balance_flag"].shift()
).cumsum()
# Create 'low_balance_streak'
eod_balance["low_balance_streak"] = eod_balance.groupby(["account_id", "streak_id"])[
    "low_balance_flag"
].cumsum()
# Create 'target'
eod_balance["target"] = (
    eod_balance["low_balance_streak"] >= incident_duration_days
) & eod_balance["low_balance_flag"]

In [40]:
# Create 'target'
bad = eod_balance.loc[
    (eod_balance.target == True) & (eod_balance.is_primary == True), "account_id"
].unique()
good = eod_balance.loc[
    (eod_balance.target == False) & (eod_balance.is_primary == True), "account_id"
].unique()

## Collect Incidents and Transactions History


In [41]:
incidents = (
    eod_balance.sort_values("balance_date")
    .loc[
        (eod_balance.low_balance_streak == incident_duration_days)
        & eod_balance.is_primary
        == True
    ]
    .sort_values("balance_date")
)

In [42]:
# Merge eod_balance with incidents_primary_ref on account_id to get incident_date for each transaction
merged = eod_balance.merge(
    incidents[["account_id", "balance_date"]],
    on="account_id",
    how="left",
    suffixes=("", "_incident"),
)
# Filter transactions that occurred within one year before their respective incident date
cashflow_bad = merged.loc[
    (
        merged["balance_date"]
        >= (merged["balance_date_incident"] - pd.DateOffset(years=1))
    )
    & (merged["balance_date"] <= merged["balance_date_incident"])
]
# Sort by balance_date
cashflow_bad = cashflow_bad.sort_values("balance_date")
cashflow_bad = cashflow_bad[
    ["account_id", "balance_date", "end_of_day_balance", "daily_amount_flow"]
]

In [43]:
cashflow_bad.sort_values("balance_date")

,account_id,balance_date,end_of_day_balance,daily_amount_flow
2732193,576,1993-01-01,900.0,900.0
2732192,576,1993-01-01,900.0,900.0
2732191,576,1993-01-01,900.0,900.0
2732190,576,1993-01-01,900.0,900.0
2732203,576,1993-01-02,900.0,0.0
...,...,...,...,...
7064934,1439,1998-12-31,19493.8,79.4
13007784,2709,1998-12-31,17459.3,74.1
1662169,349,1998-12-31,16682.4,66.7
15974854,3331,1998-12-31,16691.3,81.6


In [44]:
bad_account_ids = cashflow_bad.account_id.unique()

In [45]:
good_account_ids = eod_balance.loc[
    ~eod_balance.account_id.isin(bad_account_ids), "account_id"
].unique()

In [46]:
eod_balance.account_id.nunique()

4500

In [47]:
bad_account_ids_df = account.merge(
    pd.DataFrame(bad_account_ids, columns=["account_id"]), on="account_id"
).sort_values("account_creation_date")

In [48]:
good_account_ids_df = account.merge(
    pd.DataFrame(good_account_ids, columns=["account_id"]), on="account_id"
).sort_values("account_creation_date")

In [60]:
good_account_ids_df.loc[good_account_ids_df["account_creation_date"] < ref_date, :]

,account_id,district_id,account_statement_frequency,account_creation_date
0,3818,74,monthly,1993-01-01
1,704,55,monthly,1993-01-01
2,2378,16,monthly,1993-01-01
3,1539,1,transaction,1993-01-03
4,1695,76,monthly,1993-01-03
...,...,...,...,...
1835,2852,70,monthly,1996-12-29
1836,10404,1,monthly,1996-12-30
1837,9424,54,monthly,1996-12-30
1838,3190,1,monthly,1996-12-30


In [59]:
bad_account_ids_df.loc[bad_account_ids_df["account_creation_date"] < ref_date, :]

,account_id,district_id,account_statement_frequency,account_creation_date
0,576,55,monthly,1993-01-01
1,2632,24,monthly,1993-01-02
2,1972,77,monthly,1993-01-02
3,793,47,monthly,1993-01-03
4,2484,74,monthly,1993-01-03
...,...,...,...,...
1757,3043,1,monthly,1996-12-29
1758,528,17,monthly,1996-12-30
1759,1390,69,monthly,1996-12-30
1760,1988,72,monthly,1996-12-30
